# Euresys QuadG3

This frame grabber is newer than the Silicon Software Marathon5. 1 GB DRAM.

Using harvesters python library in `c/Work/PythonProjects/Calibration/triggered_acquisition.py`

## Connections

PCIe 3.0 (Gen 3) x4 bus: 3,300 MB/s bus bandwidth

Inside the PC there is a power connector (SATA to PCIe 6 pin PWR) so that the camera is powered by CXP.

CXP based camera trigger (from frame grabber). It can be sourced from different devices and it's controlled by ```CameraControlMethod```:
- RG : camera replies after trigger rising and waits to reply after trigger falling
- RC : camera replies rising and fallin immediately after rising. Exposure controlled by camera.
- EXTERNAL (signal triggering camera, no strobe): 
    - LIN Tool : LIN1
    - ```LineInputToolSource``` (any input in GPIO)
- NC : CXP trigger disabled (camera free run or EXT trigger)

By default, ```CxpTriggerMessageFormat``` is Pulse, meaning two CXP transactions per pulse (rising and falling) and allows controlling the exposure time. This communication means a delay between edge and exposure, this latency may be compensated by the camera.

### I/O Lines

Quad G3 has to sets of I/O Lines. For set 1:
- IIN11-4 : isolated inputs
- IOUT11-2 : isolated outputs
- DIN11-2 : RS-422
- TTLIO11-2 (LVTTL in set 2)

Outputs can drive HIGH-LOW, be open-collector or open-emitter.

### External I/O Connector

VGA connector HD-Sub 26. Counting from top right to left and down. Longer side up.

| Pin | Signal | Usage |
|---- | ------ | ----- |
| 1 | GND | Ground |
| 2 | DIN12+ | High-speed differential input #12 – Positive pole |
| 3 | IIN11+ | Isolated input #11 – Positive pole |
| 4 | IIN13- | Isolated input #13 – Negative pole |
| 5 | IIN14- | Isolated input #14 – Negative pole |
| 6 | IOUT12- | Isolated contact output #12 – Negative pole |
| 7 | GND | Ground |
| 8 | | Not connected |
| 9 | GND | Ground |
| 10 | GND | Ground |
| 11 | DIN12- | High-speed differential input #12 – Negative pole |
| 12 | IIN11- | Isolated input #11 – Negative pole |
| 13 | IIN12+ | Isolated input #12 – Positive pole |
| 14 | IIN13+ | Isolated input #13 – Positive pole |
| 15 | IIN14+ | Isolated input #14 – Positive pole |
| 16 | IOUT12+ | Isolated contact output #12 – Positive pole |
| 17 | TTLIO12 | TTL input/output #12 |
| 18 | GND | Ground |
| 19 | DIN11- | High-speed differential input #11 – Negative pole |
| 20 | DIN11+ | High-speed differential input #11 – Positive pole |
| 21 | IIN12- | Isolated input #12 – Negative pole |
| 22 | IOUT11- | Isolated contact output #11 – Negative pole |
| 23 | IOUT11+ | Isolated contact output #11 – Positive pole |
| 24 | GND | Ground |
| 25 | TTLIO11 | TTL input/output #11 |
| 26 | +12V | +12 V Power output |

### I/O Toolbox (software connections)

GPIO input may be associated to a LIN tool. This is an edge event. Can be input (DelayToolSource1-2) of a Delay Tool set by DelayToolClockSource (8NS, 200NS, 1US) and DelayToolDelayValue.

Quadrature, Divider and Multiplier/Divider : not interesting.

## Camera cycle

Two phases:
- Exposure (EXP) : \[IF asynchronous reset camera : pixel reset +\] integration + pixel transfer
- Readout (RDO) : dummy + valid pixels

Cameras not allowing overlapping -> next cycle begin after completion of readout. Shortest period : MinCyclePeriod = EXP + RDO

Cameras allowing overlapping (Optronis) -> EXP (RDO) never overlap another EXP (RDO) : MinCyclePeriod = (RDO > EXP)? EXP + RDO - EXP : EXP

Check:
- *SET ExposureReadoutOverlap in Camera Model.*
- *ExposureRecovery* : required time gap between exposures
- TargetFramePeriod : equal to RDO to achieve optimal duration of cycles in readout limited cases (max frame rate).
- CycleMinimumPeriod : always greater than RDO

Camera and Illumination Controller -> In Device Module:
- ExposureTime : duration of Camera Trigger pulse
- StrobeDuration
- StrobeDelay: with respect to camera trigger (can be negative)

*Strobe and Trigger transitions must be inside a cycle period.*

IF CycleTriggerSource == Immediate, or CycleMaxPendingTriggerCount > 0 (1-7, counter is CyclePendingTriggerCount), does not wait trigger for a new cycle. Immediate means period is CycleMinimumPeriod. Otherwise, if CycleTriggerSource == StartCycle, waits for an event.

### Sequence

StartOfSequenceTriggerSource :
- Immediate : no waiting
- StartCycle : command
- StartSequence : command
- CycleTriggerSource : command or event

EndOfSequenceTriggerSource :
- StopSequence : command
- SequenceLength : command or reached value
- Other event


## Software

Test tool command line : ```gentl ber```, then ```results```, until confidence level is maximum (100%). *NOT WORKING*

# Optronis CP80-3M-540

AcquisitionMode:
- Continuous : free run with AcquisitionFrameRate, and ExposureTime
- Single Frame : EXT, Granularity in OFF for EXT controlling exposure
- CoaxPress : exposure is between rising and falling packets if Granularity is OFF

AUX connector (square feature above, LEMO FGG.0B.304.CLAD56Z):
- UP LEFT : Sync In
- DOWN LEFT : Reserved
- UP RIGHT : Sync Out
- DOWN RIGHT : GND

Accepts a 40x40x2 mm window.

# SVS-Vistek hr120MCX

Rolling shutter is not compatible with external exposure time. The exposure time is given by the scanning time and the flash time.

Remote device:
- TriggerMode = 'Off', 'On'
- SensorTriggerMode = 'Precise', 'Fast'
- TriggerSelector = 'AcquisitionStart', 'FrameStart'
- TriggerSource = 'Software', 'Line 1 - HardwareTrigger IOMUX'
- LineSelector = 'Trigger - Line 6'
- LineSource = 'CXP Upconnection Trigger' or LensUartOut, Line 23

If more than one laser pulses are expected to occur

# Fastcam Photron Ax200

Connect ethernet cable to PC and power supply to main socket. Turn on camera. Set IP.

- Default IP: 192.168.0.10
- Port: 2000 (UDP)
- Trigger TTL IN : contact
- Input 1: SYNC POS
- Input 2: TRIG POS

Exposure time can be:
- 0.26us - 1ms : if frame rate < 1000 fps
- 0.26us - 1/fps: if frame rate > 1000 fps

Frame rate:
- max at 1Mp : 6400 fps

Images are 12bit depth.

# Test 1

## Connections

- TTLIO12 : pin 17 : Camera Trigger (for scope)
- TTLIO11 : pin 25 : Strobe
- GND : pin 24

## Configuration

### Interface

- DigitalIOControl
    - LineSelector : TTLIO11
        - LineMode : Output
        - LineSource : Device0Strobe
    - LineSelector : TTLIO12
        - LineMode : Output
        - LineSource : Device0CameraTrigger

### Device - CameraAndIlluminationControl

- CameraModel
    - CameraControlMethod : RG
    - ExposureReadoutOverlap : True
    - ExposureRecoveryTime : 10
    - CycleMinimumPeriod : 1842 (1/543)
- CycleTiming
    - ExposureTime : 300
    - StrobeDelay : 100 (laser rising edge should be within exposure time)
    - StrobeDuration : 1100 (laser requires greater than 1000us)
- CycleControl
    - CycleTriggerSource : Immediate
     
### Remote Device

- AcquisitionControl
    - AcquisitionMode : CoaxPress
    - Granularity : OFF
    
## Result

StatisticsFrameRate : 271.444 fps

# Test 2

### Device - CIC

- CameraModel
    - CameraControlMethod : RC
    
## Result

StatisticsFrameRate : 271.444 fps

# Test 3

### Device - CIC

- CameraModel
    - CameraControlMethod : NC
    
Strobe does not work. (Same with EXTERNAL)
    
### Remote Device

- AcquisitionControl
    - AcquisitionMode : Continuous
    - ExposureTime : 400
    - Granularity : OFF
    
## Results

StatisticsFrameRate : 543.071 fps

# Test 4

## Configuration

### Interface

- DigitalIOControl
    - LineSelector : TTLIO11
        - LineMode : Output
        - LineSource : Device0Strobe
    - LineSelector : TTLIO12
        - LineMode : Output
        - LineSource : Device0CameraTrigger

### Device - CIC

- CameraModel
    - CameraControlMethod : RC
    - ExposureReadoutOverlap : True
    - ExposureRecoveryTime : 1000
    - CycleMinimumPeriod : 1850 (Sets the frame rate)
- CycleTiming
    - ExposureTime: 1000 Not relevant
    - StrobeDelay : -140 (laser rising edge should be within exposure time)
    - StrobeDuration : 1100 (laser requires greater than 1000us)
- CycleControl
    - CycleTriggerSource : Immediate

### Remote Device

- AcquisitionControl
    - AcquisitionMode : CoaxPress
    - AcquisitionFrameRate : Not relevant
    - ExposureTime : 50 (us)
    - Granularity : ON
    
## Results

1. StatisticsFrameRate : 540.540 fps
2. Delay between Laser pulse (strobe) and emission : 156 us. Delay strobe should be negative and exposure time can be very short.

# SLM Meadowlarks

PCIe x8.

# Python for capture and move linear stages

## Camera (Euresys frame grabber)

### Install requirements

#### harvesters package

Installation using console in Pycharm (pip refers to python3-32bits). `pip install harvesters` gives error as requires genicam. Follow instructions from [PyPi](https://pypi.org/project/harvesters/0.post0.dev333/). Requires SVN, installing SilkSVN (downloading to Downloads folder). Packages are downloaded to `C:\PythonPackages`. `genicam` seems to require membership EMVA or similar. Deleted PythonPackages. Trying with anaconda environment with Python3.7 `conda create -n gentl_37 matplotlib python=3.7`. `conda activate gentl_37`. That's working!

#### Others

`imageio matplotlib opencv-python zaber-motion`

### PyCharm Project

`C:\Work\PythonProjects\Calibration>`. Add the anaconda environment in Settings/Project. There is a GitHub repository where this project is saved: 

# Tests for using multiple laser pulses per image

## SVS Vistek

### Test 1

Single pulse with strobe from Euresys frame grabber at 4 fps (CycleMinimumPeriod 250k). Trying different strobes delay.
- Maximum exposure : 143500
- CameraControlMethod : RC
- SensorTriggerMode : Precise
- StrobeDelay: 179500 for getting the whole image illuminated. Independent of CycleMinimumPeriod, but dependent on ExposureTime
- ExposureTime: 140000. If this goes to 130000, the StrobeDelay should go to 189500 at least.

If ExposureTime = 107k, then StrobeDelay = 212500, 107k to start getting the first rows, at 213500 the first rows start being dark. ExposureTime = 50k, StrobeDelay = 57k to start getting first rows, but the image is never complete.

If ExposureTime = 125k, in 89k starts getting first rows on.

- Starting rows: 2\*ScanningTime - ExposureTime
- All rows on: 3\*ScanningTime - ExposureTime (only for ExposureTime > ScanningTime)
- End of all rows on: 2\*ScanningTime = 213500

### Test 2

Remote device:
- ExposureTime : 115000 : All rows on : 204500
- ScanningTime : 106500
- LineSelector : Line0
- LineSource : PWM0
- Sequencer:
    - SeqInterval - interval 0 - 2\*ScanningTime (tics 15ns) : 14.2Meg
    - Seq pulse A (exposure?) start: 0 stop 14.2Meg
    - SeqPulseBStart (PWM mask) All rows on/15ns 13,633,334; 
    - SeqPulseBStop: 14.2Meg
    - PWMMax (frequency : 1000Hz) : 66667
    - PWMChange0 : 33333 (50%)
    - PWMEnable: True

## Photron

The camera allows a maximum exposure time of 1 ms so I cannot use the FTS355 to generate the laser pulses.

## Optronis

Maybe I could host to camera trigger by using divisor.

# CryLas FTS355-Q4

Trigger signal : TTL 5V pin 6 of interlock connector

# Laser Quantum Gem532

Use Putty to send commands. Serial 9600N1N. Line discipline options: Force on.

Commands:
- OFF
- ON
- POWER=##
- POWER?
- STATUS?

# Laser Leukos HORUS HLX-G-F020

The laser head connects to the controller via built-in ribbon cable. The controller connects to the connection-hub via a 16-ways flat ribbon cable. A 12V 5A Power Supply is connected to the controller using a Farnell 151866 (Molex 39-01-2020) connector.

The connection-hub has a male connector for the 16-ways flat ribbon cable. The signals that are transmitted through this cable are:
- RS232 control : 1 TX, 2 RX, 3 GND (RS232 3TX,2RX,5GND)
- interlock : 15, 16
- Modulation (external trigger COAX) : 6 IN, 7 GND.
- Pushbutton: ON/OFF : 10 IN, 11 GND
- Laser on: output signal with pulses every time the laser emits (COAX). It can be observed with the NI USB-6211 using the Oscilloscope module.

The serial interface with the laser has to be 19200bps, 8bit, 1 stop bit, no parity and no flow control. The commands to make the laser emit at maximum repetition rate are:
> *copo 1\
> *mode 1\
> *lase 1

The commands to make the laser emit with external trigger are:
> *copo 1\
> *mode 3\
> *lase 1

When using with SYNC out from the Photron camera, the delay for getting a pulse is around 34 us. The delay for the pulse should be 1/fps - 34us.